In [1]:
class DecisionTree(object):
    
    def __init__(self, max_depth=5, min_size=1, printing=False):
        self.max_depth = max_depth
        self.min_size = min_size
        self.printing = printing
    
    # Split a dataset based on an attribute and an attribute value
    def threshold_split(self, feature_index, feature_value, dataset):
        left, right = [], []
        
        #test each instance and split according to feature_value threshold
        for row in dataset:
            if row[feature_index] < feature_value:
                left.append(row)
            else:
                right.append(row)
        return left, right

    # Calculate the Gini index for a split dataset
    def gini_index(self, groups, class_values):
        #measure of purity
        #perfect class seperation: score of 0
        #worst case; 50/50 mixed classes in each group: score of 1.0
        
        gini = 0.0
        
        #for each unique class...
        for class_value in class_values:
            #for each split group...
            for group in groups:
                #check if empty
                size = len(group)
                if size == 0:
                    continue
                
                #if not empty, calculate class proportion
                proportion = [row[-1] for row in group].count(class_value) / float(size)
                
                #update gini index score
                gini += (proportion * (1.0 - proportion))
        return gini

    # Selects the best split threshold on feature/value for a dataset
    def best_split(self, dataset):
        #create list of unique classes
        class_values = list(set(row[-1] for row in dataset))
        
        #initialize best records
        b_index, b_value, b_score, b_groups = 999, 999, 999, None
        
        #for each feature (except class)...
        for feature_index in range(len(dataset[0])-1):
            #test each instance's feature value as a threshold...
            for row in dataset:
                #split into groups
                groups = self.threshold_split(feature_index, row[feature_index], dataset) #we try every feature value (row[feature_index])
                #calculate gini index score for this split
                gini = self.gini_index(groups, class_values)
                #if gini index score is better (i.e. less) than the best score so far...
                if gini < b_score:
                    b_index, b_value, b_score, b_groups = feature_index, row[feature_index], gini, groups #update best records
        return {'index':b_index, 'value':b_value, 'groups':b_groups}

    # Create a terminal node value
    def to_terminal(self, group):
        #selects the most common class value in the group to make predictions
        outcomes = [row[-1] for row in group] #actual class array
        return max(set(outcomes), key=outcomes.count) #return most common class

    # Create child splits for a node or make terminal
    def grow_tree(self, node, depth):
        #recursive procedure -- calls upon itself until threshold condition met
        
        #retrieve left/right groups
        left, right = node['groups']
        
        #delete node from memory
        del(node['groups'])
        
        # check for a no split
        if not left or not right:
            node['left'] = node['right'] = self.to_terminal(left + right)
            return
        
        # check for max depth
        if depth >= self.max_depth:
            node['left'], node['right'] = self.to_terminal(left), self.to_terminal(right)
            return
        
        # process left child
        if len(left) <= self.min_size: #check min_size
            node['left'] = self.to_terminal(left)
        else:
            node['left'] = self.best_split(left) #calculate best split
            self.grow_tree(node['left'], depth+1) #repeat until threshold condition met
        
        # process right child
        if len(right) <= self.min_size: #check min_size
            node['right'] = self.to_terminal(right)
        else:
            node['right'] = self.best_split(right) #calculate best split
            self.grow_tree(node['right'], depth+1) #repeat until threshold condition met

    # Build a decision tree
    def build_tree(self, train):
        root = self.best_split(train)
        self.grow_tree(root, 1)
        return root
    
    #trace tree for class
    def trace_tree(self, node, row):
        if row[node['index']] < node['value']:
            if isinstance(node['left'], dict):
                return self.trace_tree(node['left'], row)
            else:
                return node['left']
        else:
            if isinstance(node['right'], dict):
                return self.trace_tree(node['right'], row)
            else:
                return node['right']
    
    # Classification and Regression Tree Algorithm
    def fit(self, trainSet, testSet):
        self.testSet = testSet
        
        tree = self.build_tree(trainSet)
        return(tree)
        
    #predict class
    def predict(self,node):
        
        predictions = []
        
        for row in self.testSet:
            prediction = self.trace_tree(node, row)
            predictions.append(prediction)
        
        return(predictions)
    
   #evaluate predictions
    def score(self, predictions):
        correct = sum(predictions == pd.DataFrame(self.testSet)[9]) 
        accuracy = correct / len(self.testSet)
        return accuracy
            
    

In [2]:
import numpy as np
import pandas as pd

In [3]:
#load data
df = pd.read_csv('../data/breast-cancer-wisconsin.data.txt')

#preprocess data
df = df.replace('?',-99999) #not vulnerable to outliers
df = df.astype(float)
df = df.drop(['id'],1)

#Shuffle the Data
df = df.reindex(np.random.permutation(df.index))
df = df.reset_index(drop=True)

#preview data
df.head()

,clump_thickness,uni_cell_size,uni_cell_shape,marg_adhes,epith_cell_size,bare_nuclei,bland_chrom,norm_nucleoli,mitoses,class
0,2.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,1.0,2.0
1,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0
2,6.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
3,4.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0
4,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0


In [4]:
#create copy
tf = df.copy()
#tf = tf.drop('class',axis=1) #drop class

#feature names
features = tf.columns

#Define the percentage of the data that you want to use for testing
test_size = 0.2

#Grabs the first (1-test_size) of the data
train_data = tf[:-int(test_size*len(tf))]

#Grabs the last (test_size) of the data
test_data = tf[-int(test_size*len(tf)):]

#save as value arrays
train_data = train_data.values
test_data = test_data.values

In [5]:
cart = DecisionTree()
tree = cart.fit(train_data,test_data)
predictions = cart.predict(tree)
cart.score(predictions)

0.95683453237410077